In [1]:
import math

def emo_lev(arousal,valence):    #Function that returns emotion and intensity
    arousal -= 5
    valence -= 5
    r = 5
    x, y = valence, arousal
    dist = (x**2+y**2)**0.5

    #Calc theta in radian
    theta = math.atan(y/x)

    #Convert theta from rad to deg
    theta = 180 * theta/math.pi

    def lev(dist):
        if(dist<1.67):
            return "Low"
        if(dist>1.67 and dist<3.34):
            return "Medium"
        else:
            return "High"
        
    #Check which intensity
    levl = lev(dist)


    def conv_emo(theta):
        if(x>0 and y>0):  #1st Quadrant
            if theta <= 90:
                if(theta<45):
                    return "Happy",1.1,theta  #1st Quad, 1st half
                else:
                    return "Excited",1.2,theta #1st Quad, 2nd half
        else:
            if(x>0 and y<0): #4
                theta+=360
                quad = 4
                if theta <= 360:
                    if(theta<315):
                        return "Calm",4.1,theta
                    else:
                        return "Content",4.2,theta
            else:
                if(x<0 and y>0): #2
                    theta+=180
                    quad = 2
                    if theta <= 180:
                        if(theta<135):
                            return "Angry",2.1,theta
                        else:
                            return "Afraid",2.2,theta
                else:
                    if(x<0 and y<0):
                       theta+=180
                       if theta <= 270:
                            if(theta<225):
                                return "Sad",3.1,theta
                            else:
                                return "Depressed",3.2,theta
                       
                   
    #print("Theta before conversion: {}".format(theta))
    emo,quad,theta = conv_emo(theta)
    #print("Distance = {} \nTheta after conversion = {} \nquadrant = {} ".format(dist,theta,quad))
    return emo,levl,theta,quad    #END of emo_lev

#Input the valence, arousal values here.
valence_x, arousal_y = 2,3
emotion, level, theta,quadrant = emo_lev(arousal_y,valence_x)

#Optional printing. Just return the values you need if you want.
print("Emotion    : "+emotion)
print("Intensity  : "+level)
print("Theta      : {} degree".format("%.2f"%theta))
print("Quadrant   : {}".format(quadrant))

Emotion    : Sad
Intensity  : High
Theta      : 213.69 degree
Quadrant   : 3.1


In [6]:
from models.tcn import EEG_TCN
from dataset.DEAP_DATASET import ModularDeapDataset
from torch.utils.data import DataLoader
import torch
from util.train import select_last_elm

DATA_SET_PATH = 'dataset/'
deap_val = ModularDeapDataset(DATA_SET_PATH, train=False)
loader = DataLoader(deap_val, shuffle=True, batch_size=1)

INPUT_SIZE = 40
HIDDEN_SIZE1 = 64
HIDDEN_SIZE2 = 32
OUTPUT_SIZE = 4

MDL_TCN_PATH = 'models/saved_weights/tcn_deeper_v2.pth'

CHAN_LIST = [32, 24, 16, 10, 6, 2]
KERN_SIZE = 5
DROP_OUT = 0.2

model = EEG_TCN(CHAN_LIST,KERN_SIZE,DROP_OUT)

model.load_state_dict(torch.load(MDL_TCN_PATH, map_location="cpu"))
model.eval()

with torch.no_grad():
    for i, (signal, label) in enumerate(loader):
        output = model(signal)
        output = select_last_elm(output, "cpu")
        output = output.numpy()
        label = label.numpy()
        print("----------------------------------")

        emotion, level, theta,quadrant = emo_lev(label[0][1],label[0][0])
        print("ACTUAL",label)
        print("\tEmotion    : "+emotion)
        print("\tIntensity  : "+level)
        print("\tTheta      : {} degree".format("%.2f"%theta))
        print("\tQuadrant   : {}".format(quadrant))
        
        emotion, level, theta,quadrant = emo_lev(output[0][1],output[0][0])
        print("PREDICTED",output)
        print("\tEmotion    : "+emotion)
        print("\tIntensity  : "+level)
        print("\tTheta      : {} degree".format("%.2f"%theta))
        print("\tQuadrant   : {}".format(quadrant))
        



----------------------------------
ACTUAL [[8.96 7.12]]
	Emotion    : Happy
	Intensity  : High
	Theta      : 28.16 degree
	Quadrant   : 1.1
PREDICTED [[5.0075307 3.6634884]]
	Emotion    : Calm
	Intensity  : Low
	Theta      : 270.32 degree
	Quadrant   : 4.1
----------------------------------
ACTUAL [[7.44 3.73]]
	Emotion    : Content
	Intensity  : Medium
	Theta      : 332.50 degree
	Quadrant   : 4.2
PREDICTED [[9.681251 8.718658]]
	Emotion    : Happy
	Intensity  : High
	Theta      : 38.46 degree
	Quadrant   : 1.1
----------------------------------
ACTUAL [[5.01 7.97]]
	Emotion    : Excited
	Intensity  : Medium
	Theta      : 89.81 degree
	Quadrant   : 1.2
PREDICTED [[6.2687116 8.199869 ]]
	Emotion    : Excited
	Intensity  : High
	Theta      : 68.37 degree
	Quadrant   : 1.2
----------------------------------
ACTUAL [[5.04 4.96]]
	Emotion    : Content
	Intensity  : Low
	Theta      : 315.00 degree
	Quadrant   : 4.2
PREDICTED [[4.4443374 9.483262 ]]
	Emotion    : Angry
	Intensity  : High
	Th

----------------------------------


TypeError: cannot unpack non-iterable NoneType object